In [ ]:
import sys
sys.path.append('../src')
import numpy as np
import pandas as pd
import tensorflow as tf
import reproducibility
from metrics import rmse

## Model Description

The `input_shape` argument specifies (`time_steps`, `n_features`)

## Data Processing

In [ ]:
# Get one timeseries of fuel moisture data
df = pd.read_pickle("../data/rocky_2023_05-09.pkl")
# First STID from unique
st = df.stid.unique()[0]
fm = df.fm[df.stid == st].to_numpy()
print(f"Station: {st}")
print(f"N observations: {len(fm)}")

h2 = int(len(fm)*.9) # index for train/test split
train = fm[0:h2]
test = fm[h2:len(fm)]

print(f"Length Train: {train.shape[0]}")
print(f"Length Test: {test.shape[0]}")

## RNN

In [ ]:
input_dim = train.shape[0]
print(input_dim)

In [ ]:
reproducibility.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(
        units = 1, 
        input_shape=(None, 1), 
        activation="linear"),
    tf.keras.layers.Dense(units=1, activation="linear")
])
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Check Model Weights on Initialization
wx = model.get_weights()[0]
wh = model.get_weights()[1]
bh = model.get_weights()[2]
wy = model.get_weights()[3]
by = model.get_weights()[4]
 
print('wx = ', wx, ' wh = ', wh, ' bh = ', bh, ' wy =', wy, 'by = ', by)

In [ ]:
# Reset Model Weights to match autoreg
W_x = np.array([[0.97427477]])  # Input weight beta_1
W_h = np.array([[0.0]])  # Recurrent weights beta_1?
b = np.array([0.23754674])      # Bias aka mean beta_0

model.layers[0].set_weights([W_x, W_h, b])

model.layers[1].set_weights([np.array([[1]]), np.array([0.0]) ])


wx = model.get_weights()[0]
wh = model.get_weights()[1]
bh = model.get_weights()[2]
wy = model.get_weights()[3]
by = model.get_weights()[4]
 
print('wx = ', wx, ' wh = ', wh, ' bh = ', bh, ' wy =', wy, 'by = ', by)

In [ ]:
# Predict Test
nsteps = len(test) # number of steps into the future to forecast
y_current = train[-1] # latest train observation, use to step model forward

forecast = []
current_input = y_current

for _ in range(nsteps):
    # Reshape the input data to match the model's input shape
    current_input = np.array(current_input).reshape(1, 1, 1)
    
    # Predict the next data point
    next_pred = model.predict(current_input)
    
    # Append the predicted data point to the forecast
    forecast.append(next_pred[0, 0])  # Assuming your model outputs a single value
    
    # Update input data for the next prediction
    current_input = np.roll(current_input, -1)
    current_input[-1] = next_pred
    
# def rnn_forecast(model, y_current, n):
#     preds = []
#     y_current = np.array(y_current).reshape(1, 1, 1)
#     next_pred = model.predict(y_current)
#     preds.append(next_pred)
#     return

In [ ]:
print(f"RMSE Test: {rmse(forecast, test)}")

## References

https://machinelearningmastery.com/understanding-simple-recurrent-neural-networks-in-keras/